In [102]:
from foundations.projectables import Projectable
from foundations.projectables import restructure
from foundations.projectables import feedback_residuals
from foundations.projectables import strategy_eliminate_feedfwd
from modeling.compute import create_vars, Par, Var
from foundations.functional_noobj import eliminate_vars
from foundations.functional_noobj import concatenate_residuals
from foundations.functional_noobj import optimizer_solver
from foundations.functionals import encode_sympy
from modeling.execution import sympy_to_edges
from graph.graphutils import default_tree
from presolver.tearprepare import execute_tearing
from graph.operators import reformulate
from numpy import pi

In [7]:
A,V,S,W = create_vars('A V S W')
t, W0 = 0.12, 4940
Struct = Projectable()
Ww = Struct.Var('Ww', 45.42*S+8.71e-5*2.5/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2))
Struct.add_equation(W, W0+Ww)
functional = Struct.functional(x_initial={W:1000})
functional.dict_in_dict_out({A:10, S:10}, cleanup=True)

{W: 6415.7614501678045, Ww: 1475.761450167805}

In [8]:
Df, Ds, Dd, tf, ts, td, hf, mtot = create_vars('Df Ds Dd tf ts td hf m_{platform}')
g, rhow = 9.81, 1023.6

Hydro = Projectable()
Vd = Hydro.Var('Vd', pi/4*(Df**2*hf+Ds**2*ts+Dd**2*td))
FB = Hydro.Var('F_B', rhow*Vd*g)
FW = Hydro.Var('F_W', FB)
xd,xs = td/2,td+ts/2
xf1,xf2 = td+ts+hf/2, td+ts+tf/2
totA1, totA2 = hf*Df+ts*Ds+td*Dd, tf*Df+ts*Ds+td*Dd
KB = Hydro.Var('K_B', (hf*Df*xf1+ts*Ds*xs+td*Dd*xd)/totA1) 
KG = Hydro.Var('K_G', (tf*Df*xf2+ts*Ds*xs+td*Dd*xd)/totA2) 
I = Hydro.Var('I', pi/64*Df**4)
BM = Hydro.Var('B_M', I/Vd)
#should be 3-5% according to http://web.mit.edu/13.012/www/handouts/Reading3.pdf
GM = Hydro.Var('G_M', KB+BM-KG) 
C33 = Hydro.Var('C_{33}', pi*rhow*g/4*Df**2)
A33 = Hydro.Var('A_{33}', 0.0525*pi*rhow*(Dd**3+Ds**3+Df**3))
omega0 = Hydro.Var('\omega_0', (C33/(A33+mtot))**1/2)

In [9]:
mbatt, A_s = create_vars('m_{batt} A_s')
mprop, mcomms, eta_solar, rho, rhoh = 50, 50, 10, 700, 2700
Mass = Projectable()
Mass.add_equation(mtot, FW/g)
msolar = Mass.Var('m_{solar}', eta_solar*A_s)
mstruct = Mass.Var('m_{struct}', mtot-mbatt-msolar-mcomms-mprop) 
Mass.add_equation(td, (4/pi*mstruct-Df**2*tf*rho-Ds**2*ts*rho)/(Dd**2*rhoh))

In [79]:
yH = {Df:2, Ds:2, Dd:2, tf:0.1, ts:0.1, td:0.1, hf:0.9*0.3, mtot:800}
HF = Hydro.functional()
yM = {mbatt:100, A_s:0.5, FW:36130}
MF = Mass.functional()

In [88]:
merged_eqs = Hydro.sympy_equations+Mass.sympy_equations
F, R, solvevars = feedback_residuals(merged_eqs)

In [103]:
combined = strategy_eliminate_feedfwd(R,F,solvevars, 
                           x_initial={mtot:1000, td:0.01})

In [112]:
combined.encoder.order

(Df, tf, Dd, Ds, ts, hf, A_s, m_{batt})

In [123]:
yH = {Df:2, Ds:2, Dd:2, tf:0.1, ts:0.1, hf:0.9, mbatt:0, A_s:0.5}

In [124]:
combined.dict_in_dict_out({**yH, **yM})

{m_{platform}: 4785.5175703586165,
 td: 0.48815704684581535,
 Vd: 4.675183245758693,
 F_B: 46945.927365217845,
 F_W: 46945.927365217845,
 K_B: 0.7440785234229077,
 K_G: 0.3440785234229077,
 I: 0.785398163397448,
 B_M: 0.16799302232911575,
 G_M: 0.5679930223291157,
 C_{33}: 31546.35289650436,
 A_{33}: 4051.825142670288,
 \omega_0: 1.7848324955190167,
 m_{solar}: 5.0,
 m_{struct}: 4580.5175703586165}

In [98]:
Relim = eliminate_vars(R, F)

In [100]:
Relim.dict_in_flat_out({**yH, **yM})

array([-7.11395093e+02,  8.17057843e-02])

In [86]:
'{:.2f} %'.format(HF.dict_in_dict_out(yH)[BM]*100)
#MF.dict_in_dict_out({**yM, **yH})

'53.19 %'

In [13]:
edges = sympy_to_edges(merged_eqs, tvar=None,
                       filterto=lambda x: not x.always_input)

In [32]:
not_input = [omega0, A33, BM, KG, I, mstruct, KB, msolar]
not_output = [Df, Dd, Ds, ts, tf, td]

In [33]:
xout = execute_tearing(edges, not_input=not_input, not_output=not_output)

In [34]:
outset_initial = {key:val[0] for key,val in edges[1].items()}
_, new_tree = reformulate(edges, default_tree(edges[0].keys()), edges[1], xout)

In [35]:
new_eqs = restructure(merged_eqs, xout)

In [36]:
new_eqs_order = [new_eqs[idx] for idx in new_tree[0].keys()]

In [37]:
F,R,solvervars = feedback_residuals(new_eqs_order)

In [43]:
Rsys = concatenate_residuals((Hydro.residuals(), 
                             Mass.residuals()))

In [47]:
F.encoder.order

(Df, Ds, Dd, tf, td, ts, A_s, m_{batt})

In [59]:
y = {Df:2, Ds:1, Dd:1.25, tf:0.5, ts:2, td:0.5, A_s:0.5, mbatt:100}

In [62]:
eliminate_vars(obj, F).dict_in_flat_out(y)

array([4060.81410843])

In [48]:
obj  = encode_sympy(mtot)
ineq = concatenate_residuals((
    encode_sympy(hf-0.9*tf),
    encode_sympy(Ds-0.9*Df),
    encode_sympy(Ds-0.9*Dd),
    encode_sympy(0.1-td)))
eq = R

In [53]:
eliminate_vars(obj, F)

() <- (Df, Ds, Dd, tf, td, ts, A_s, m_{batt})

In [63]:
obj_el, ineq_el = (eliminate_vars(obj, F), 
                    eliminate_vars(ineq, F))

In [69]:
OPT = optimizer_solver(obj_el, ineqs=(ineq_el,), bounds={
    Df:(.1,10), Ds:(.1,10), Dd:(.1,10), 
    tf:(.1,10), ts:(.1,10), td:(.1,10), A_s:(.1,10), mbatt:(.1,1000)})

In [70]:
OPT.dict_in_dict_out(x0=y)

c:\Users\johan\miniconda3\envs\minimdo\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


{Df: 0.22802724650406425,
 Ds: 0.20522431714601252,
 Dd: 0.22802713886610262,
 tf: 0.10000000039645938,
 td: 0.10000000214464105,
 ts: 9.999999998247896,
 A_s: 0.10000000068502242,
 m_{batt}: 0.10000000005000768}